In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as ss

In [2]:
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', None)

In [3]:
# #get rid of scientific notation
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [4]:
# failed sound check 1957441

# Fish

In [6]:
# import file
data = pd.read_csv('data_exp_27270-v5_task-d7fe.csv', sep = ",")
# data_2 = pd.read_csv("pilot_2/data_exp_24005-v8_task-lhx6.csv", sep = ",")


In [7]:
data.task.unique()

array([nan, 'fishing_control', 'mood_rating_control'], dtype=object)

In [8]:
data = data.replace('null', np.nan)
data  = data.replace('nan', np.nan)


In [9]:
data['start_time'] = round(data['start_time'].astype(float))

In [10]:
data = data[['Participant Private ID', 'UTC Timestamp','key pressed', 'price_displayed', 'environment', 'task', 'function id',
       'builtin rt', 'end_time', 'start_time', 'response', 'round_instance']]


In [11]:
data.rename(columns={"Participant Private ID": "Subject"}, inplace = True)


In [12]:
data = data.sort_values(by = ['Subject', 'start_time'])

In [13]:
data['trial_label'] = data[['environment', 'round_instance']].apply(lambda x: ' '.join(x.astype(str)), axis=1)


In [14]:
# exclude
# data = data[data.Subject != 1972447]

In [15]:
data["environment_binary"] = np.where(data["environment"] == 'pre_mood', 0, 1)

In [16]:
data["mood"] = data["task"].map({"mood_rating_pos":"pos",
                                    "mood_rating_neg":"neg",
                                'mood_rating_control':'control'})


In [17]:
data = data.sort_values(by = ["Subject", "UTC Timestamp"])


In [18]:
data.reset_index(inplace = True)

In [19]:
data.Subject.nunique()

25

# Latency

In [20]:
fish = data[(data.task == "fishing_pos")|(data.task == "fishing_neg")|(data.task == "fishing_control")]


In [21]:
fish = fish[fish['start_time'].notna()]

In [22]:
fish['start_time'] = fish['start_time'].astype(int)

In [23]:
fish.environment.unique()

array(['pre_mood', 'post_mood'], dtype=object)

In [24]:
d = fish.groupby(['Subject','environment_binary', 'round_instance'])['start_time'].diff().shift(-1).reset_index(name = 'latency')
d = d.set_index('index')


In [25]:
fish['latency'] = d['latency']
fish = fish.sort_index().sort_values(['Subject', 'start_time'])

In [26]:
def mask_first(x):
    """
    get rid of the first spike by creating a mask of 0, 1, 1, 1, ...
    """
    result = np.ones_like(x)
    result[0] = 0
    return result

In [27]:
mask = fish.groupby(['Subject', 'environment'])['round_instance'].transform(mask_first).astype(bool)

# data_useful = data_useful.loc[mask]


In [28]:
fish = fish.loc[mask]

In [33]:
fish.head()
fish["mood"] = fish["task"].map({'fishing_pos': "pos", 'fishing_neg':'neg',
                                'fishing_control':'control'})


In [34]:
fish.groupby("mood")["Subject"].nunique()

mood
control    25
Name: Subject, dtype: int64

In [35]:
fish['mood'].unique()

array(['control'], dtype=object)

In [36]:
fish.latency.isnull().values.any()
fish = fish[fish.latency.notnull()]

In [37]:
# mad
fish.shape

(45799, 17)

In [38]:
median = np.median(fish.latency)
low = median - 3*ss.median_absolute_deviation(fish.latency)
above = median + 3*ss.median_absolute_deviation(fish.latency)
fish = fish[(fish.latency>=low)&(fish.latency<=above)]

In [39]:
# mad
fish.shape

(43845, 17)

# Rating

In [40]:
# response
rating = data[(data.task == "mood_rating_pos")|(data.task == "mood_rating_neg")|(data.task == "mood_rating_control")]



In [41]:
rating = rating[rating['response'].notna()]

# Concat 

In [42]:
rating["latency"] = np.nan*len(rating)

In [43]:
data_complete = pd.concat([rating, fish])


In [44]:
data_complete = data_complete.sort_values(by = "index")

# Backfilling

In [45]:
data_complete["response"].bfill(inplace = True)

# df.bfill(axis ='rows')


In [46]:
data_complete["block"] = data_complete["round_instance"].map({
    1:"1st",
    2:"1st",
    3:"1st",
    4:"1st",
    5:"2nd",
    6:"2nd",
    7:"2nd",
    8:"2nd",
    9:"3rd",
    10:"3rd",
    11:"3rd",
    12:"3rd"
})

In [47]:
data_complete.to_csv("data_control_no_feedback_latency_backfilled_happiness.csv")
